In [64]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder

In [65]:
images=[]
globalpath='C:\\Users\\jerar\\OneDrive\\Desktop\\food_classify\\train\\Train'
os.chdir(globalpath)
print(os.getcwd())
labels=os.listdir()#list of all foodnames since food name is the directory name
print(labels)
class_len=len(labels)
print(class_len)

C:\Users\jerar\OneDrive\Desktop\food_classify\train\Train
['Baked_Potato', 'Burger', 'Crispy_Chicken', 'Donut', 'Fries', 'Hot_Dog', 'Pizza', 'Sandwich', 'Taco', 'Taquito']
10


In [66]:
data={"image":[],
    "label":[]
}
for label in labels:
    os.chdir(globalpath)
    path=os.getcwd()+"\\"+label
    os.chdir(path)
    imagelist=os.listdir()
    labellist=[]
    for i in range(0,len(imagelist)):
        imagelist[i]=path+"\\"+imagelist[i]
        labellist.append(label)
    data["image"].extend(imagelist)
    data["label"].extend(labellist)


'''
data["label"]=list(encoded_labels)'''
#load data into a DataFrame object:
df = pd.DataFrame(data)
print(df.head())


                                               image         label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...  Baked_Potato
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...  Baked_Potato
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...  Baked_Potato
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...  Baked_Potato
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...  Baked_Potato


In [67]:
#encoding labels
encoder = LabelEncoder()
encoded_labels= encoder.fit_transform(df.iloc[:,1])
df.iloc[:,1]=encoded_labels
print(df.head())


                                               image label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0


In [68]:
### creating tensors from dictionary using tf.data.Dataset.from_tensor_slices(tensors)
### tensors should be 1D arrays
###

In [69]:
images=df.iloc[:,0:1].values #using 0:1 since even though there is single column , X should be 2d and y should be 1D
labels=df.iloc[:,1].values
X_train, X_test, y_train, y_test=train_test_split(images, labels, test_size=0.05)

In [70]:
def preprocess(imagepath, label):
        file = tf.io.read_file(imagepath)  # Read image file
        image = tf.image.decode_jpeg(file, channels=3)  # Decode JPEG file into an image
        image = tf.image.resize(image, [128, 128])  # Resize to 128x128
        image = image / 255.0  # Normalize to [0, 1]
        image = tf.image.random_flip_left_right(image) # random flipping
        image = tf.image.random_brightness(image, max_delta=0.1) # random change in brightness
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2) # random change in contrast
        return image, label

In [71]:
train_data=tf.data.Dataset.from_tensor_slices((list(X_train[:,0]), list(y_train))).map(preprocess)
test_data= tf.data.Dataset.from_tensor_slices((list(X_test[:,0]), list(y_test))).map(preprocess)

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu'),   # Increased from 128 to 256
    Dropout(0.5),                    # Added Dropout to prevent overfitting
    Dense(class_len, activation='softmax')
])


In [73]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  #  'categorical_crossentropy' if labels are one-hot encoded
    metrics=['accuracy']
)
batch_size = 32
train_dataset = train_data.batch(batch_size)
model.fit(train_dataset, epochs=25)

Epoch 1/25
439/439 [==============================] - 99s 224ms/step - loss: 2.4643 - accuracy: 0.1511
Epoch 2/25
439/439 [==============================] - 100s 227ms/step - loss: 2.2092 - accuracy: 0.1802
Epoch 3/25
439/439 [==============================] - 99s 225ms/step - loss: 2.1180 - accuracy: 0.2218
Epoch 4/25
439/439 [==============================] - 98s 223ms/step - loss: 2.0621 - accuracy: 0.2475
Epoch 5/25
439/439 [==============================] - 99s 226ms/step - loss: 1.9850 - accuracy: 0.2738
Epoch 6/25
439/439 [==============================] - 100s 228ms/step - loss: 1.9554 - accuracy: 0.2910
Epoch 7/25
439/439 [==============================] - 100s 229ms/step - loss: 1.8944 - accuracy: 0.3176
Epoch 8/25
439/439 [==============================] - 101s 230ms/step - loss: 1.8090 - accuracy: 0.3564
Epoch 9/25
439/439 [==============================] - 100s 228ms/step - loss: 1.7372 - accuracy: 0.3923
Epoch 10/25
439/439 [==============================] - 99s 226ms/ste

In [74]:
print(type(train_dataset)) # should be (num_samples, 128, 128, 3)


<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [75]:
test_data = test_data.batch(batch_size)
loss, accuracy = model.evaluate(test_data)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

24/24 [==============================] - 3s 87ms/step - loss: 1.6301 - accuracy: 0.6184
Test loss: 1.6300982236862183
Test accuracy: 0.6184032559394836


In [76]:
print(class_len)

10
